# Neural Networks and Deep Learning A.Y. 2022/2023
## Final Project: 3D Objects Classification - PointNet model

Group members:<br>
* *Candon Matteo 2020353*: ICT for Internet and Multimedia (Cybersystems) <br>
* *Nicoletti Gianpietro 2053042*: ICT for Internet and Multimedia (Cybersystems) <br>
* *Rizzetto Nicola 2052417*: Computer Engineering (Artificial Intelligence and Robotics) <br>

## Initial configuration and import libraries

In [26]:
#Platform settings: this is an automatinc decision so, please, don't change the value
COLAB = True

try:
    from google.colab import drive
except:
    COLAB = False
    

#Dataset settings:
LOAD_DATASET = True #True if you want to load a precomputed dataset 
ALREADY_DOWNLOADED = False #True if you have already the dataset in the memory
ALREADY_UNZIPPED = False #True if you have already unzipped the dataset in the ./data folder, IF True the value of
                        #ALREADY_DOWNLOADED variable will be ignore

#Model settings:
LOAD_MODEL = False #True if you want load the trained model for XX epochs

In [27]:
PURPLE = '\033[95m'
CYAN = '\033[96m'
DARKCYAN = '\033[36m'
BLUE = '\033[94m'
GREEN = '\033[92m'
YELLOW = '\033[93m'
RED = '\033[91m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
END = '\033[0m'

In [28]:
#if working with colab these are needed
if (COLAB):
    !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
    !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
    !pip install git+https://github.com/pyg-team/pytorch_geometric.git


    !pip install tensorflow
    !pip install trimesh
    !pip install open3d

import trimesh # useful to load polygonal objects (off files)
import glob # useful to read filenames with specific patterns, i.e. chair_****
#import open3d

# Neural Networks libraries
import tensorflow as tf
from tensorflow import keras
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#from torchmetrics.classification import MulticlassAccuracy
#from torch_geometric.nn import GCNConv,global_max_pool
#from torch.optim.lr_scheduler import StepLR

#graph libraries
from sklearn.neighbors import kneighbors_graph
import networkx as nx

import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
import time # used to check the training duration
import math
from sklearn.utils import shuffle

#manual set of seed for reproducibility
seed = 353
tf.random.set_seed(seed)

from tqdm import tqdm, trange
import gc

import os
import collections
import urllib.request
import zipfile

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(BOLD+"You are working on: "+END+str(device))

You are working on: cpu


## Utility functions definition

In [30]:
#usefull functions

cloud_points = 2048 #number of points to describe an object

def dataset(points = cloud_points):
    train_points = []
    train_labels = []
    test_points = []
    test_labels = []
    classes_map = {}
    
    #saving all classes' folder paths in list
    folders = [dir for dir in sorted(os.listdir(data_dir))if os.path.isdir(data_dir)]
    folders_path=[]
    for i in range(10):
        folders_path.append(glob.glob(os.path.join(data_dir, folders[i])))
        
        
    for i,folder in enumerate(folders_path):
        
        #storing classes' names into list
        
        print(i)
        print(folder[0])
        print(type(folder[0]))
        
        classes_map[i] = folder[0].split("\\")[-1]
        
        #storing train and test files
        train_files = glob.glob(os.path.join(folder[0], "train/*"))
        test_files = glob.glob(os.path.join(folder[0], "test/*"))
        
        #converting train and test files in cloud points
        for j in train_files:
            train_points.append(trimesh.load(j).sample(cloud_points))
            train_labels.append(i)

        for j in test_files:
            test_points.append(trimesh.load(j).sample(cloud_points))
            test_labels.append(i)

    return (np.array(train_points),
            np.array(test_points),
            np.array(train_labels),
            np.array(test_labels),
            classes_map)

def build_graph(data_points):
    
    edges = []
    
    for i in trange(len(data_points)):
        
        A = kneighbors_graph(data_points[i],mode='distance', n_neighbors = 50).toarray()
        indices = np.argpartition(A, 50, axis=1)[:, :50]
        edge_indexes = np.stack([np.repeat(np.arange(data_points[i].shape[0]), 50), indices.flatten()])
#         A = nx.from_numpy_array(A)
#         edges.append(np.asarray(list(A.edges())).T)
        edges.append(edge_indexes)
        print(edge_indexes.shape)
        
        del A
        del edge_indexes
        del indices
        gc.collect()
    
    return edges

## Creating or loading the dataset

2048 points: https://www.dropbox.com/s/t5h5nmphju4ni14/dataset_nndl_2048.zip?dl=1 <br>

**NOTES:**
* dl=1 : force the download of the file;
* In order to select the correct dataset you need only to modify the url in the method `urlretrieve(url,name)`
* Remember to change the value of `cloud_points` according to the datasets you want use

In [31]:
if(LOAD_DATASET):
    
    cloud_points = 2048
    
    if(not ALREADY_DOWNLOADED and (not ALREADY_UNZIPPED)):
        urllib.request.urlretrieve("https://www.dropbox.com/s/t5h5nmphju4ni14/dataset_nndl_2048.zip?dl=1", "dataset_nndl.zip")
        
    if(not ALREADY_UNZIPPED):
        with zipfile.ZipFile("dataset_nndl.zip", 'r') as zip_ref:
            zip_ref.extractall("./data/")
        
    #loading the data
    data_train = np.load("data/train_data.npy",allow_pickle=True)
    data_test = np.load("data/test_data.npy",allow_pickle=True)
    edge_train = np.load("data/train_edges.npy",allow_pickle=True)
    edge_test = np.load("data/test_edges.npy",allow_pickle=True)
    train_labels = np.load("data/train_labels.npy",allow_pickle=True)
    test_labels = np.load("data/test_labels.npy",allow_pickle=True)

In [32]:
if(not LOAD_DATASET):
    data_dir = tf.keras.utils.get_file("modelnet.zip", "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip", extract = True)
    data_dir = os.path.join(os.path.dirname(data_dir), "ModelNet10")

#each class is already divided in train and test data

In [33]:
#Visualization of data example
if(not LOAD_DATASET):
    mesh = trimesh.load(os.path.join(data_dir,"chair/train/chair_0003.off"))
    mesh.show()

In [34]:
if(not LOAD_DATASET):
    #setting parameters
    classes = 10 #number of different kind of objects - non modifiable

    # Creating dataset object
    data_start_time = time.time()
    train_points, test_points, train_labels, test_labels, classes_map = dataset(cloud_points)
    data_end_time = time.time()

    print("Time spent creating the dataset object: ",np.round((data_end_time - data_start_time)/60,2), " minutes.")

In [35]:
if(not LOAD_DATASET):
    
    print(len(train_points))
    print(len(test_points))

    #data_train = add_normals(train_points)
    #data_test = add_normals(test_points)

    edge_train=build_graph(train_points)
    edge_test=build_graph(test_points)

    
    #removing usless variables to clean the RAM
    del train_points
    del test_points
    gc.collect()

### Saving the dataset into local memory

**NOTES:**

If you want create a link to be used in this code, compress the .npy files into a single .zip file.<br>
Make sure that the structure of the .zip will be:<br>
<br>
folder.zip<br>
--file1.npy<br>
--file2.npy<br>
-- ...<br>

<br>
and not something like:<br>
<br>
folder.zip<br>
--inner folder<br>
----file1.npy<br>
----file2.npy<br>
---- ...
<br>
<br>
Mantain these names:

* list of points for the train: train_data.npy
* list of points for the test: test_data.npy
* list of edges for the train: train_edges.npy
* list of edges for the test: test_edges.npy
* labels for the train: train_labels.npy
* labels for the test: test_labels.npy

In [36]:
if(not LOAD_DATASET):
    #list of arrays to array of arrays
    train = np.empty(len(edge_train), object)
    train[:] = edge_train
    test = np.empty(len(edge_test), object)
    test[:] = edge_test
    
    #saving the data
    np.save("data/train_data.npy",data_train)
    np.save("data/test_data.npy",data_test)
    np.save("data/train_edges.npy",train)
    np.save("data/test_edges.npy",test)
    np.save("data/train_labels.npy",train_labels)
    np.save("data/test_labels.npy",test_labels)

## Shuffle and data augmentation

In [37]:
torch.cuda.empty_cache()
gc.collect()

123

In [38]:
#shuffle of the original data
indexes =[i for i in range(len(train_labels))]
import random
random.shuffle(indexes)

data_train = data_train[indexes]
train_labels = train_labels[indexes]
#edge_train = edge_train[indexes]

In [39]:
#data augmentation: i.e. adding noise to the data(poistion + normal) but mainaining the same graph and label.
# in this case we obtain 3x samples of the original dataset
data_train_noise = np.concatenate((data_train,data_train+normal(0,5,size = (len(data_train),2048,6))),axis=0)
train_labels_noise = np.concatenate((train_labels,train_labels),axis=0)
#edge_train_noise =  np.concatenate((edge_train,edge_train),axis=0)

In [40]:
#shuffle of the augmented dataset
indexes =[i for i in range(len(train_labels_noise))]
import random
random.shuffle(indexes)

data_train_noise = data_train_noise[indexes]
train_labels_noise = train_labels_noise[indexes]
#edge_train_noise = edge_train_noise[indexes]

## Network functions and model

In [41]:
class Tnet(nn.Module):
    def __init__(self, k=3):
        super().__init__()
        self.k=k
        self.conv1 = nn.Conv1d(k,64,1)
        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
        self.fc1 = nn.Linear(1024,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,k*k)

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
        self.bn4 = nn.BatchNorm1d(512)
        self.bn5 = nn.BatchNorm1d(256)


    def forward(self, input):
        # input.shape == (bs,n,3)
        bs = input.size(0)
        xb = F.relu(self.bn1(self.conv1(input)))
        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = F.relu(self.bn3(self.conv3(xb)))
        pool = nn.MaxPool1d(xb.size(-1))(xb)
        flat = nn.Flatten(1)(pool)
        xb = F.relu(self.bn4(self.fc1(flat)))
        xb = F.relu(self.bn5(self.fc2(xb)))

        #initialize as identity
        init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
        if xb.is_cuda:
            init=init.cuda()
        matrix = self.fc3(xb).view(-1,self.k,self.k) + init
        return matrix

In [42]:
class Transform(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)


        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)

    def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64


In [43]:
class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

In [44]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

In [45]:
# parameters
#cloud_points = 2048 # Number of points to describe an object
#classes = 40 # Number of classes in the dataset - set for ModelNet40
#batch_size = 32
#learning_rate = 0.00025
#dropout = 0.5
#num_epochs = 50

In [46]:
pointnet = PointNet()
pointnet.to(device)

PointNet(
  (transform): Transform(
    (input_transform): Tnet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feature_transform): Tnet(
      (conv1): Conv1d(64, 64, kernel_size=(1,

## Training

In [47]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.00025)

In [57]:
def train(model, data_train_loader, labels_train_loader, val_loader=None,  epochs=4):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(data_train_loader, 0):
            inputs = data_train_loader[:][:,0:3]
            labels = labels_train_loader 
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 5 == 4:    # print every 10 mini-batches
                print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                    (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                running_loss = 0.0

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)


In [58]:
train_labels_noise

array([1, 8, 5, ..., 7, 5, 5])

In [59]:
#print(data_train_noise)
train(model = pointnet, data_train_loader= data_train_noise, labels_train_loader = train_labels_noise)

ValueError: axes don't match array

In [ ]:
torch.cuda.empty_cache()
gc.collect()